In [1]:
import parselmouth as ps
from parselmouth.praat import call as pcall  
import audiolabel
import pandas as pd

This notebook assumes that these files are located in the same directory as the notebook. If your files are in another location, or if you want to use different audiofiles, you can point your script to new places.

In [2]:
breathyfile = './21_06_25_TikTok_14_mono.wav'
modalfile = './21_6_25_mestres_modal.wav'
breathytest = ps.Sound(breathyfile)
modaltest = ps.Sound(modalfile)

- For this notebook, there is one new TextGrid, '21_06_25_TikTok_14.TextGrid', which will supplement the old TextGrid (which has the vowels marked in it).
- The TextGrid '21_6_25_mestres_modal.TextGrid' has been modified to include the two additional tiers that appear in the other TextGrids, indicating where the post-IP break occurs (SP) and any notable voice quality changes during the lead up to the IP break.

In [3]:
tg_breathy = audiolabel.LabelManager(from_file='./21_06_25_TikTok_14_mono.TextGrid', from_type='praat')
tg_breathy_adtl = audiolabel.LabelManager(from_file='./21_06_25_TikTok_14.TextGrid', from_type='praat') # this is the new one
tg_modal = audiolabel.LabelManager(from_file='./21_6_25_mestres_modal.TextGrid', from_type='praat') # this one has been altered

As before, we'll use the `Parselmouth` to do calculate HNR. (Nothing new here!)

In [4]:
h_breathy = breathytest.to_harmonicity_cc()
h_modal = modaltest.to_harmonicity_cc()

We'll use a method from `audiolabel` on `Label` objects, `as_df`, to turn the TextGrid files into lists of `Pandas` dataframes. 

This first TG is the same as the one we worked with last week, and has only one tier. So it is a list with a single dataframe.

In [5]:
tg_breathy.as_df()

[          t1         t2 text   duration     center
 0   0.000000  10.306042       10.306042   5.153021
 1  10.306042  10.449255    4   0.143213  10.377648
 2  10.449255  10.540974        0.091719  10.495114
 3  10.540974  10.584227    3   0.043253  10.562600
 4  10.584227  10.700680        0.116454  10.642454
 5  10.700680  10.742117    2   0.041437  10.721399
 6  10.742117  10.888221        0.146104  10.815169
 7  10.888221  10.946052    1   0.057831  10.917137
 8  10.946052  19.930979        8.984927  15.438516]

This next one is the supplemental TG for the same breathy audio file, and as you can see here it is a list of two dataframes, one for each tier. 

In [6]:
tg_breathy_adtl.as_df()

[          t1         t2 text   duration     center
 0   0.000000  11.173687       11.173687   5.586844
 1  11.173687  12.354081  SP    1.180394  11.763884
 2  12.354081  19.930979        7.576898  16.142530,
           t1         t2     text   duration     center
 0   0.000000  10.873734           10.873734   5.436867
 1  10.873734  11.173687  whisper   0.299953  11.023711
 2  11.173687  19.930979            8.757292  15.552333]

Note that there's no information about the names of the tiers here though. So it's actually probably easier to track what's going on if we first access the tier, and _then_ turn it into a dataframe. Like so:

In [7]:
tg_breathy_adtl.tier('whisper').as_df()

,t1,t2,text,duration,center
0,0.000000,10.873734,,10.873734,5.436867
1,10.873734,11.173687,whisper,0.299953,11.023711
2,11.173687,19.930979,,8.757292,15.552333


Now, let's create a starting dataframe from the vowels tier of the breathy textgrid.

In [8]:
breathydf = tg_breathy.tier('vowels').as_df()
breathydf

,t1,t2,text,duration,center
0,0.000000,10.306042,,10.306042,5.153021
1,10.306042,10.449255,4,0.143213,10.377648
2,10.449255,10.540974,,0.091719,10.495114
3,10.540974,10.584227,3,0.043253,10.562600
4,10.584227,10.700680,,0.116454,10.642454
5,10.700680,10.742117,2,0.041437,10.721399
6,10.742117,10.888221,,0.146104,10.815169
7,10.888221,10.946052,1,0.057831,10.917137
8,10.946052,19.930979,,8.984927,15.438516


Notice though that this includes a bunch of intervals that are unlabeled. We don't want these though, so let's remove them.

In [9]:
breathydf = breathydf[breathydf.text!='']
breathydf

,t1,t2,text,duration,center
1,10.306042,10.449255,4,0.143213,10.377648
3,10.540974,10.584227,3,0.043253,10.562600
5,10.700680,10.742117,2,0.041437,10.721399
7,10.888221,10.946052,1,0.057831,10.917137


You may want to be able to add other information that is contained either in the names of the files (e.g. speaker ID, or recording date) or in the annotations in the TextGrid (e.g. vowel quality, voice quality, word ID, etc). 

Here is how you can access information in the labels of the TextGrids at a given time.

In [10]:
tg_breathy_adtl.labels_at(11) # This returns a "named tuple" of labels at time 11s

Ret(text=Label( t1=0.0000, t2=11.1737, text='b''' ), whisper=Label( t1=10.8737, t2=11.1737, text='b'whisper'' ))

The information in a "named tuple" can be accessed in several different ways, but one of them is by the names, like so. As you can see, this returns the Label object that had the name 'whisper'. (Here, 'whisper' is from the name of the tier, not the content of the label.) 

In [11]:
tg_breathy_adtl.labels_at(11).whisper 

Label( t1=10.8737, t2=11.1737, text='b'whisper'' )

This behaves just like a regular `Label` object, because it is a `Label` object! So we can access its time points as well as its text label.

In [12]:
tg_breathy_adtl.labels_at(11).whisper.text

'whisper'

So now we can use the `apply` function in `Pandas` to get the information on the "whisper" tier of the second textgrid in the breathy example for each of the vowels in our vowels dataframe, and add it to a new column of the dataframe. (Ignore the red Warning; it's not an Error, just a warning.)

In [13]:
breathydf['voice'] = breathydf.center.apply(lambda t: tg_breathy_adtl.labels_at(t).whisper.text)
breathydf

<ipython-input-13-d3754edc3cc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breathydf['voice'] = breathydf.center.apply(lambda t: tg_breathy_adtl.labels_at(t).whisper.text)


,t1,t2,text,duration,center,voice
1,10.306042,10.449255,4,0.143213,10.377648,
3,10.540974,10.584227,3,0.043253,10.562600,
5,10.700680,10.742117,2,0.041437,10.721399,
7,10.888221,10.946052,1,0.057831,10.917137,whisper


We may also want filename information in there, like this. There are many other bits of information you could put directly into the filename and then extract and put into the dataframe.

In [14]:
breathydf['file'] = breathyfile
breathydf

<ipython-input-14-9baefbc8265c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breathydf['file'] = breathyfile


,t1,t2,text,duration,center,voice,file
1,10.306042,10.449255,4,0.143213,10.377648,,./21_06_25_TikTok_14_mono.wav
3,10.540974,10.584227,3,0.043253,10.562600,,./21_06_25_TikTok_14_mono.wav
5,10.700680,10.742117,2,0.041437,10.721399,,./21_06_25_TikTok_14_mono.wav
7,10.888221,10.946052,1,0.057831,10.917137,whisper,./21_06_25_TikTok_14_mono.wav


In [15]:
modaldf = tg_modal.tier('vowels').as_df()
modaldf = modaldf[modaldf.text!='']
modaldf['voice'] = modaldf.center.apply(lambda t: tg_modal.labels_at(t).whisper.text)
modaldf['file'] = modalfile
modaldf

,t1,t2,text,duration,center,voice,file
1,7.730573,7.801286,4,0.070713,7.765930,,./21_6_25_mestres_modal.wav
3,7.884021,7.969774,3,0.085753,7.926898,,./21_6_25_mestres_modal.wav
5,8.094046,8.161063,2,0.067017,8.127555,,./21_6_25_mestres_modal.wav
7,8.279621,8.390241,1,0.110620,8.334931,creaky,./21_6_25_mestres_modal.wav


Now, we can use the `apply` function again, combined with the Parselmouth commands we learned previously.

In [16]:
breathydf['hnr_mid']=breathydf.center.apply(lambda t: h_breathy.get_value(t))
breathydf['hnr_mean']=breathydf.apply(lambda v: pcall(h_breathy, 'Get mean...', v.t1, v.t2), axis=1)
breathydf

<ipython-input-16-5f82c21c59ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breathydf['hnr_mid']=breathydf.center.apply(lambda t: h_breathy.get_value(t))
<ipython-input-16-5f82c21c59ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  breathydf['hnr_mean']=breathydf.apply(lambda v: pcall(h_breathy, 'Get mean...', v.t1, v.t2), axis=1)


,t1,t2,text,duration,center,voice,file,hnr_mid,hnr_mean
1,10.306042,10.449255,4,0.143213,10.377648,,./21_06_25_TikTok_14_mono.wav,15.822534,8.789439
3,10.540974,10.584227,3,0.043253,10.562600,,./21_06_25_TikTok_14_mono.wav,9.453398,7.446303
5,10.700680,10.742117,2,0.041437,10.721399,,./21_06_25_TikTok_14_mono.wav,21.441211,4.346415
7,10.888221,10.946052,1,0.057831,10.917137,whisper,./21_06_25_TikTok_14_mono.wav,-200.000000,NaN


In [17]:
modaldf['hnr_mid']=modaldf.center.apply(lambda t: h_modal.get_value(t))
modaldf['hnr_mean']=modaldf.apply(lambda v: pcall(h_modal, 'Get mean...', v.t1, v.t2), axis=1)
modaldf

,t1,t2,text,duration,center,voice,file,hnr_mid,hnr_mean
1,7.730573,7.801286,4,0.070713,7.765930,,./21_6_25_mestres_modal.wav,17.883930,14.413955
3,7.884021,7.969774,3,0.085753,7.926898,,./21_6_25_mestres_modal.wav,25.477782,22.037220
5,8.094046,8.161063,2,0.067017,8.127555,,./21_6_25_mestres_modal.wav,14.880742,10.698104
7,8.279621,8.390241,1,0.110620,8.334931,creaky,./21_6_25_mestres_modal.wav,11.753899,7.102502


Now we can put together the dataframes into a single dataframe.

In [18]:
alldf = modaldf.append(breathydf)
alldf

,t1,t2,text,duration,center,voice,file,hnr_mid,hnr_mean
1,7.730573,7.801286,4,0.070713,7.765930,,./21_6_25_mestres_modal.wav,17.883930,14.413955
3,7.884021,7.969774,3,0.085753,7.926898,,./21_6_25_mestres_modal.wav,25.477782,22.037220
5,8.094046,8.161063,2,0.067017,8.127555,,./21_6_25_mestres_modal.wav,14.880742,10.698104
7,8.279621,8.390241,1,0.110620,8.334931,creaky,./21_6_25_mestres_modal.wav,11.753899,7.102502
1,10.306042,10.449255,4,0.143213,10.377648,,./21_06_25_TikTok_14_mono.wav,15.822534,8.789439
3,10.540974,10.584227,3,0.043253,10.562600,,./21_06_25_TikTok_14_mono.wav,9.453398,7.446303
5,10.700680,10.742117,2,0.041437,10.721399,,./21_06_25_TikTok_14_mono.wav,21.441211,4.346415
7,10.888221,10.946052,1,0.057831,10.917137,whisper,./21_06_25_TikTok_14_mono.wav,-200.000000,NaN


Finally, we may want to save the dataframe to a CSV file or otherwise, so we can more easily use it in another application or share it with others.

In [19]:
alldf.to_csv('./2_data.csv')